# 如何搭建一个 RAG 项目

## 项目背景

首先要有一份可以作为 RAG 项目的数据集，且该数据集可以支撑我们实现一个问答系统。

以本文举例，我们使用的是 [HowToCook](https://github.com/Anduin2017/HowToCook) 数据集，包含了大约300多个Markdown格式的菜谱文件。

这些文件有两个关键特点：

- 结构高度规整，每个文件都严格按照统一的格式来组织内容。

- 内容篇幅较短，单个菜谱通常在700字左右。

这种文件不需要过多处理就可以直接用于RAG系统构建。

## 搭建项目

### 数据准备模块

RAG系统的效果很大程度上取决于数据准备的质量。

在该项目中，我们发现每个菜谱文件的内容都不算太长，单个章节的内容通常在几百字左右，这意味着可以直接按照标题进行分块。

但是严格按照标题分块会把内容切得太细，导致上下文信息不完整，LLM就无法给出完整的制作指导。

如果直接把整个菜谱文档作为一个块，可以发现效果反而比结构分块要好，因为上下文信息是完整的。

为了解决这个矛盾，可以采用**父子文本块**的策略：用小的子块进行精确检索，但在生成时传递完整的父文档给LLM。

**父子文本块映射关系**

父文档（完整菜谱）

- 菜品介绍 + 难度评级
- 必备原料和工具
- 计算（用量配比）
- 操作（制作步骤）
- 附加内容（变化做法）

**基本流程**

1. 检索阶段：使用小的子块进行精确匹配，提高检索准确性。
2. 生成阶段：传递完整的父文档给LLM，确保上下文完整性。
3. 智能去重：当检索到同一道菜的多个子块时，合并为一个完整菜谱。

**元数据增强**

- 菜品分类：从文件路径推断（荤菜、素菜、汤品等）
- 难度等级：从内容中的星级标记提取
- 菜品名称：从文件名提取
- 文档关系：建立父子文档的ID映射关系

#### 模块实现

##### 加载markdown 文件

功能：

- 批量读取并加载所有 markdown 文件。
- 每个 markdown 文件都作为父文档分配唯一 ID。
- 为每个父文档创建 Document 对象
- 根据文档内容添加元数据。

In [1]:
import uuid
from pathlib import Path
from langchain_core.documents import Document

documents = []

data_path_obj = Path("data/cook")

# 遍历目录下（包含子目录）所有 markdown 文件
for md_file in data_path_obj.rglob("*.md"):
    # 读取整个文件的内容，并赋值给 content 变量
    with open(md_file, 'r', encoding='utf-8') as f:
            content = f.read()
    
    # 为每个父文档创建唯一 ID
    parent_id = str(uuid.uuid4())
    
    # 创建 Document 对象
    doc = Document(
        page_content=content, # 父文档的内容
        metadata={
            "source": str(md_file), # 父文档的来源路径
            "parent_id": parent_id, # 父文档的唯一 ID
            "doc_type": "parent" # 文档类型，标记为父文档
        } # 父文档的元数据
    )
    # 将父文档添加到文档列表中
    documents.append(doc)
    
print(documents[0])


page_content='# 咖喱炒蟹的做法

第一次吃咖喱炒蟹是在泰国的建兴酒家中餐厅，爆肉的螃蟹挂满有蟹黄味道的咖喱，味道真的绝，喜欢吃海鲜的程序员绝对不能错过。操作简单，对沿海的程序员非常友好。

预估烹饪难度：★★★★

## 必备原料和工具

- 青蟹（别称：肉蟹）
- 咖喱块（推介乐惠蟹黄咖喱）
- 洋葱
- 椰浆
- 鸡蛋
- 生粉（别称：淀粉）
- 大蒜

## 计算

每次制作前需要确定计划做几份。一份正好够 1 个人食用

总量：

- 肉蟹 1 只（大约 300g） * 份数
- 咖喱块 15g（一小块）*份数
- 椰浆 100ml*份数
- 鸡蛋 1 个 *份数
- 洋葱 200g *份数
- 大蒜 5 瓣 *份数

## 操作

- 肉蟹掀盖后对半砍开，蟹钳用刀背轻轻拍裂，切口和蟹钳蘸一下生粉，不要太多。撒 5g 生粉到蟹盖中，盖住蟹黄，备用
- 洋葱切成洋葱碎，备用
- 大蒜切碎，备用
- 烧一壶开水，备用
- 起锅烧油，倒入约 20ml 食用油，等待 10 秒让油温升高
- 将螃蟹切口朝下，轻轻放入锅中，煎 20 秒，这一步主要是封住蟹黄，蟹肉。然后翻面，每面煎 10 秒。煎完将螃蟹取出备用
- 将螃蟹盖放入锅中，使用勺子舀起锅中热油泼到蟹盖中，煎封住蟹盖中的蟹黄，煎 20 秒后取出备用
- 不用刷锅，再倒入 10ml 食用油，大火让油温升高至轻微冒烟，将大蒜末，洋葱碎倒入，炒 10 秒钟
- 将咖喱块放入锅中炒化（10 秒），放入煎好的螃蟹，翻炒均匀
- 倒入开水 300ml，焖煮 3 分钟。
- 焖煮完后，倒入椰浆和蛋清，关火，关火后不断翻炒，一直到酱汁变浓稠。
- 出锅

## 附加内容

- 做法参考：[十几年澳门厨房佬教学挂汁的咖喱蟹怎么做](https://www.bilibili.com/video/BV1Nq4y1W7K9)' metadata={'source': 'data\\cook\\dishes\\aquatic\\咖喱炒蟹.md', 'parent_id': '8b15b1b4-56e4-4a05-a4af-7d1c531c3107', 'doc_type': 'parent'}


##### 增强元数据

- 分类推断: 从HowToCook项目的目录结构推断菜品分类。
- 难度提取: 从内容中的星级标记自动提取难度等级。
- 名称提取: 直接使用文件名作为菜品名称。

In [2]:
# 定义菜品分类
category_mapping = {
        'meat_dish': '荤菜', 'vegetable_dish': '素菜', 'soup': '汤品',
        'dessert': '甜品', 'breakfast': '早餐', 'staple': '主食',
        'aquatic': '水产', 'condiment': '调料', 'drink': '饮品'
    }

# 遍历父文档
for doc in documents:
    
    # 获取文件路径
    file_path = Path(doc.metadata.get('source', ''))
    # 拆分为各个组成部分
    path_parts = file_path.parts
    
    # 从路径中推断分类
    doc.metadata['category'] = '其他'
    for key, value in category_mapping.items():
        if key in file_path.parts:
            doc.metadata['category'] = value
            break
    
    # 提取菜品名称
    # 提取不带扩展名的文件名
    doc.metadata['dish_name'] = file_path.stem
    
    # 分析难度等级
    content = doc.page_content
    if '★★★★★' in content:
        doc.metadata['difficulty'] = '非常困难'
    elif '★★★★' in content:
        doc.metadata['difficulty'] = '困难'
    elif '★★★' in content:
        doc.metadata['difficulty'] = '中等'
    elif '★★' in content:
        doc.metadata['difficulty'] = '简单'
    elif '★' in content:
        doc.metadata['difficulty'] = '非常简单'
    else:
        doc.metadata['difficulty'] = '未知'

print(documents[0])


page_content='# 咖喱炒蟹的做法

第一次吃咖喱炒蟹是在泰国的建兴酒家中餐厅，爆肉的螃蟹挂满有蟹黄味道的咖喱，味道真的绝，喜欢吃海鲜的程序员绝对不能错过。操作简单，对沿海的程序员非常友好。

预估烹饪难度：★★★★

## 必备原料和工具

- 青蟹（别称：肉蟹）
- 咖喱块（推介乐惠蟹黄咖喱）
- 洋葱
- 椰浆
- 鸡蛋
- 生粉（别称：淀粉）
- 大蒜

## 计算

每次制作前需要确定计划做几份。一份正好够 1 个人食用

总量：

- 肉蟹 1 只（大约 300g） * 份数
- 咖喱块 15g（一小块）*份数
- 椰浆 100ml*份数
- 鸡蛋 1 个 *份数
- 洋葱 200g *份数
- 大蒜 5 瓣 *份数

## 操作

- 肉蟹掀盖后对半砍开，蟹钳用刀背轻轻拍裂，切口和蟹钳蘸一下生粉，不要太多。撒 5g 生粉到蟹盖中，盖住蟹黄，备用
- 洋葱切成洋葱碎，备用
- 大蒜切碎，备用
- 烧一壶开水，备用
- 起锅烧油，倒入约 20ml 食用油，等待 10 秒让油温升高
- 将螃蟹切口朝下，轻轻放入锅中，煎 20 秒，这一步主要是封住蟹黄，蟹肉。然后翻面，每面煎 10 秒。煎完将螃蟹取出备用
- 将螃蟹盖放入锅中，使用勺子舀起锅中热油泼到蟹盖中，煎封住蟹盖中的蟹黄，煎 20 秒后取出备用
- 不用刷锅，再倒入 10ml 食用油，大火让油温升高至轻微冒烟，将大蒜末，洋葱碎倒入，炒 10 秒钟
- 将咖喱块放入锅中炒化（10 秒），放入煎好的螃蟹，翻炒均匀
- 倒入开水 300ml，焖煮 3 分钟。
- 焖煮完后，倒入椰浆和蛋清，关火，关火后不断翻炒，一直到酱汁变浓稠。
- 出锅

## 附加内容

- 做法参考：[十几年澳门厨房佬教学挂汁的咖喱蟹怎么做](https://www.bilibili.com/video/BV1Nq4y1W7K9)' metadata={'source': 'data\\cook\\dishes\\aquatic\\咖喱炒蟹.md', 'parent_id': '8b15b1b4-56e4-4a05-a4af-7d1c531c3107', 'doc_type': 'parent', 'category': '水产', 'dish_name': '咖喱炒蟹', 'difficulty': '困难'}


##### 结构分块

将完整的菜谱文档按照Markdown标题结构进行分块，实现父子文本块架构。

分块逻辑：

- 子块1: 包含一级标题及其下的所有内容（简介、难度评级），直到遇到下一个二级标题
- 子块2-5: 每个二级标题及其下的内容形成一个独立子块
- 精确检索: 用户问"需要什么食材"时，能精确匹配到子块2
- 上下文完整: 生成时传递完整的父文档，包含所有必要信息

In [3]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

# 定义要分割的标题层级
headers_to_split_on = [
    ("#", "主标题"),      # 菜品名称
    ("##", "二级标题"),   # 必备原料、计算、操作等
    ("###", "三级标题")   # 简易版本、复杂版本等
]

# 创建 markdown 分割器
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False # 保留标题，便于理解上下文
)

all_chunks = []
parent_child_map = {}

for doc in documents:
    # 对父文档进行分割
    md_chunks = markdown_splitter.split_text(doc.page_content)
    
    parent_id = doc.metadata["parent_id"]
    for i, chunk in enumerate(md_chunks):
        child_id = str(uuid.uuid4())
        chunk.metadata.update({
            "chunk_id": child_id, # 子文档的唯一 ID
            "parent_id": parent_id, # 子文档的父文档 ID
            "doc_type": "child", # 文档类型，标记为子文档
            "chunk_index": i # 在父文档中的位置
        }) # 子文档的元数据
        
        # 建立父子映射关系
        parent_child_map[child_id] = parent_id
    
    all_chunks.extend(md_chunks)

chunks = all_chunks

# 添加元数据
for i, chunk in enumerate(chunks):
    if 'chunk_id' not in chunk.metadata:
        # 如果没有chunk_id（比如分割失败的情况），则生成一个
        chunk.metadata['chunk_id'] = str(uuid.uuid4())
    chunk.metadata['batch_index'] = i  # 在当前批次中的索引
    chunk.metadata['chunk_size'] = len(chunk.page_content) # 子文档的内容长度
    
for i in range(6):
    print(chunks[i].metadata)



{'主标题': '咖喱炒蟹的做法', 'chunk_id': 'd3efaea5-e94f-47bb-a58e-400eb77be27f', 'parent_id': '8b15b1b4-56e4-4a05-a4af-7d1c531c3107', 'doc_type': 'child', 'chunk_index': 0, 'batch_index': 0, 'chunk_size': 102}
{'主标题': '咖喱炒蟹的做法', '二级标题': '必备原料和工具', 'chunk_id': '1cd7ecb9-5e6a-4518-9a67-691a9ff17d9c', 'parent_id': '8b15b1b4-56e4-4a05-a4af-7d1c531c3107', 'doc_type': 'child', 'chunk_index': 1, 'batch_index': 1, 'chunk_size': 72}
{'主标题': '咖喱炒蟹的做法', '二级标题': '计算', 'chunk_id': 'abf382b0-4807-47d3-b171-8d2df62bf4b2', 'parent_id': '8b15b1b4-56e4-4a05-a4af-7d1c531c3107', 'doc_type': 'child', 'chunk_index': 2, 'batch_index': 2, 'chunk_size': 138}
{'主标题': '咖喱炒蟹的做法', '二级标题': '操作', 'chunk_id': 'b56aa170-09d9-41ed-90f5-3673f33dbf38', 'parent_id': '8b15b1b4-56e4-4a05-a4af-7d1c531c3107', 'doc_type': 'child', 'chunk_index': 3, 'batch_index': 3, 'chunk_size': 390}
{'主标题': '咖喱炒蟹的做法', '二级标题': '附加内容', 'chunk_id': '07138038-fd83-4f52-b0f6-6450f2ac9f09', 'parent_id': '8b15b1b4-56e4-4a05-a4af-7d1c531c3107', 'doc_type': '

##### 智能去重

同一道菜可能会检索到多个子块，所以我们需要智能去重，避免重复信息。

去重逻辑：

- 统计相关性: 计算每个父文档被匹配的子块数量
- 按相关性排序: 匹配子块越多的菜谱排名越靠前
- 去重输出: 每个菜谱只输出一次完整文档


In [4]:
from typing import List

def get_parent_documents(child_chunks: List[Document]) -> List[Document]:
    """根据子块获取对应的父文档（智能去重）"""
    # 统计每个父文档被匹配的次数（相关性指标）
    parent_relevance = {}
    parent_docs_map = {}

    # 收集所有相关的父文档ID和相关性分数
    for chunk in child_chunks:
        parent_id = chunk.metadata.get("parent_id")
        if parent_id:
            # 统计每个 parent_id 被出现的次数。
            parent_relevance[parent_id] = parent_relevance.get(parent_id, 0) + 1

            # 缓存父文档（避免重复查找）
            if parent_id not in parent_docs_map:
                for doc in documents:
                    if doc.metadata.get("parent_id") == parent_id:
                        parent_docs_map[parent_id] = doc
                        break

    # 按相关性排序并构建去重后的父文档列表
    sorted_parent_ids = sorted(parent_relevance.keys(), key=lambda x: parent_relevance[x], reverse=True)

    # 构建去重后的父文档列表
    parent_docs = []
    for parent_id in sorted_parent_ids:
        if parent_id in parent_docs_map:
            parent_docs.append(parent_docs_map[parent_id])

    return parent_docs


数据准备模块详细代码：[date_preparation](rag_modules/date_preparation.py)


### 索引构建模块

将文本块转换为向量表示，并构建高效的检索索引。

使用的BGE-small-zh-v1.5作为嵌入模型，并使用FAISS作为向量数据库来存储和检索向量。

为了提升系统启动速度，实现索引缓存机制。

首次构建后会将FAISS索引保存到本地，后续启动时直接加载已有索引，可以将启动时间从几分钟缩短到几秒钟。

#### 初始化嵌入模型

In [5]:
# 初始化嵌入模型

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
        model_name="../models/bge/bge-small-zh-v1.5",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )


#### 构建索引

使用FAISS作为向量数据库。

In [6]:
# 构建索引

from langchain_community.vectorstores import FAISS

if not chunks:
    raise ValueError("文档块列表不能为空")

# 提取文本内容
texts = [chunk.page_content for chunk in chunks]
metadatas = [chunk.metadata for chunk in chunks]

# 构建FAISS向量索引
vectorstore = FAISS.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas
)


#### 索引缓存机制

In [ ]:
# 索引缓存机制

if not vectorstore:
    raise ValueError("请先构建向量索引")

index_save_path = "vector_index"

Path(index_save_path).mkdir(parents=True, exist_ok=True)

vectorstore.save_local(index_save_path)


NameError: name 'index_save_path' is not defined

In [8]:
# 从本地加载向量索引

from langchain_community.vectorstores import FAISS

index_save_path = "vector_index"

vectorstore = FAISS.load_local(
    index_save_path, 
    embeddings,
    allow_dangerous_deserialization=True
)


索引构建模块详细代码：[index_construction](rag_modules/index_construction.py)

### 检索优化模块

采用双路检索的方式：

- 向量检索基于语义相似度，擅长理解查询意图。
- BM25检索基于关键词匹配，擅长精确匹配。

使用RRF算法来融合检索结果。

系统还支持基于元数据的智能过滤，可以按菜品分类、难度等级等条件进行筛选检索。

#### 检索器设置

In [9]:
# 检索器设置

from langchain_community.retrievers import BM25Retriever

# 向量检索器
vector_retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )

# BM25检索器
bm25_retriever = BM25Retriever.from_documents(
    chunks,
    k=5
)


#### RRF 混合检索

In [10]:
query = "西红柿"

vector_docs = vector_retriever.invoke(query)
bm25_docs = bm25_retriever.invoke(query)

print(vector_docs)
print("************")
print(bm25_docs)


[Document(id='c50a3384-6bbe-4929-bd2b-019f3a5f43f8', metadata={'主标题': '西葫芦炒鸡蛋的做法', '二级标题': '操作', 'chunk_id': '4c34fe85-f37e-47ad-9bc5-f414eb670690', 'parent_id': 'b4d22003-b554-483e-9db7-e489511b9fc7', 'doc_type': 'child', 'chunk_index': 3, 'batch_index': 1742, 'chunk_size': 234}, page_content='## 操作  \n- 西红柿洗净，切成小块，备用\n- 西葫芦洗净，切成边长约为 4cm 的菱形，备用\n- 打三个鸡蛋到碗里，打散搅匀，备用\n- 热锅，锅内放入 5ml - 10ml 食用油\n- 倒入鸡蛋，保持翻炒至鸡蛋成固体，用锅铲分成小块后盛到碗里，备用\n- 锅内放入 5ml - 10ml 食用油，倒入西红柿，炒至变软\n- 倒入西葫芦一起翻炒均匀，放入 6g 食用盐，将火调小然后**等待 4 - 5 分钟**\n- 倒入备用的鸡蛋，中火翻炒 15 秒\n- 关火，盛盘'), Document(id='19258ae0-fd44-4e3f-83a7-fe501e221325', metadata={'主标题': '糖拌西红柿的做法', '二级标题': '必备原料和工具', 'chunk_id': '2134b723-080e-4b59-8c71-fbd736216f97', 'parent_id': '8837406a-66eb-4b1a-91db-70b2f668850f', 'doc_type': 'child', 'chunk_index': 1, 'batch_index': 1695, 'chunk_size': 29}, page_content='## 必备原料和工具  \n- 西红柿\n- 白砂糖\n- 冰箱'), Document(id='81bca3ee-6d72-4c1d-a6c0-6f93668f9bed', metadata={'主标题': '糖拌西红柿的做法', '二级标题': '操作', 'chunk_id': '78a824b6-6442-4

In [11]:
# 使用 RRF 重排

rrf_scores = {}
k = 60 # RRF 重排参数

# 计算向量检索的RRF分数
for rank, doc in enumerate(vector_docs):
    doc_id = id(doc)
rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + 1 / (k + rank + 1)

# 计算BM25检索的RRF分数
for rank, doc in enumerate(bm25_docs):
    doc_id = id(doc)
    rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + 1 / (k + rank + 1)
    
# 合并所有文档并按RRF分数排序
all_docs = {id(doc): doc for doc in vector_docs + bm25_docs}
sorted_docs = sorted(all_docs.items(),
                    key=lambda x: rrf_scores.get(x[0], 0),
                    reverse=True)

reranked_docs = [doc for _, doc in sorted_docs]

print(reranked_docs[:5])


[Document(metadata={'主标题': '糖拌西红柿的做法', '二级标题': '必备原料和工具', 'chunk_id': '88cbb167-2418-43d9-a2a1-d2897c1c49a5', 'parent_id': '74f1c823-3c99-4324-9585-908b4afafd97', 'doc_type': 'child', 'chunk_index': 1, 'batch_index': 1695, 'chunk_size': 29}, page_content='## 必备原料和工具  \n- 西红柿\n- 白砂糖\n- 冰箱'), Document(metadata={'主标题': '西红柿牛腩的做法', '二级标题': '计算', 'chunk_id': 'fbfe704b-1e70-4dd1-ba96-9bd8e205882a', 'parent_id': '5c7b8ba7-f661-4895-bcb5-bd02b819969c', 'doc_type': 'child', 'chunk_index': 2, 'batch_index': 1308, 'chunk_size': 59}, page_content='## 计算  \n每份：  \n- 西红柿 3-4 个（每个约 200g）\n- 牛腩 500g\n- 食用油 20-30ml'), Document(metadata={'主标题': '西红柿炒鸡蛋的做法', '二级标题': '必备原料和工具', 'chunk_id': '947d7463-0718-4f87-b391-01b656f95c7f', 'parent_id': 'd360d6aa-ea59-4c30-9a94-b2ee86e0c77f', 'doc_type': 'child', 'chunk_index': 1, 'batch_index': 751, 'chunk_size': 50}, page_content='## 必备原料和工具  \n* 西红柿\n* 鸡蛋\n* 食用油\n* 盐\n* 糖（可选）\n* 葱花（可选）'), Document(metadata={'主标题': '西红柿鸡蛋汤的做法', '二级标题': '必备原料和工具', 'chunk_id': '38314

In [ ]:
# 从子文档获取父文档

parent_docs = get_parent_documents(reranked_docs)
parent_docs


[Document(metadata={'source': 'data\\cook\\dishes\\vegetable_dish\\糖拌西红柿\\糖拌西红柿.md', 'parent_id': '74f1c823-3c99-4324-9585-908b4afafd97', 'doc_type': 'parent', 'category': '素菜', 'dish_name': '糖拌西红柿', 'difficulty': '简单'}, page_content='# 糖拌西红柿的做法\n\n![示例菜成品](./火山飘雪.jpg)\n\n新鲜可口，制作简便，营养价值高，适合夏季食用，家庭餐桌上的一道美味凉菜。西红柿含有大量的维生素 C, 做法简单 几分钟就可完成。\n\n预估烹饪难度：★★\n\n## 必备原料和工具\n\n- 西红柿\n- 白砂糖\n- 冰箱\n\n## 计算\n\n一份正好够 2 人吃。\n\n每份：\n\n- 西红柿 2 个（每个西红柿约 100g，共 200g）\n- 白砂糖 20g\n\n## 操作\n\n- 用刀将西红柿皮米字型划开\n- 用筷子插入西红柿的菊花，在燃气上转动烤 10 秒（或用开水冲 30 秒），直到西红柿皮卷边\n- 把西红柿的衣服脱光\n- 再西红柿大卸八块（沿纹路切可以更多的留汁水），去掉头部根蒂部，备用\n- 全部切好后，将西红柿在盘子中均匀码一层\n- 撒上白糖，重复上面一步直到全部西红柿放完\n- 放入冰箱冷藏 10 分钟\n- 一盘糖拌西红柿就好了，营养美味，酸甜爽口，夏日解暑又解腻\n\n## 附加内容\n\n在制作过程中 请您小心使用刀具。'),
 Document(metadata={'source': 'data\\cook\\dishes\\meat_dish\\西红柿牛腩\\西红柿牛腩.md', 'parent_id': '5c7b8ba7-f661-4895-bcb5-bd02b819969c', 'doc_type': 'parent', 'category': '荤菜', 'dish_name': '西红柿牛腩', 'difficulty': '非常困难'}, page_content='# 西红柿牛腩的做法\n\n西红柿牛腩汤汁浓厚酸甜可口，牛肉软绵醇香，搭配米饭绝配。一般初学者需要 9

### 元数据过滤检索

In [15]:
filters={"主标题": "糖拌西红柿的做法"}

vector_retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 15, "filter": filters}  # 扩大检索范围
    )

results = vector_retriever.invoke(query)

print(results)


[Document(id='19258ae0-fd44-4e3f-83a7-fe501e221325', metadata={'主标题': '糖拌西红柿的做法', '二级标题': '必备原料和工具', 'chunk_id': '2134b723-080e-4b59-8c71-fbd736216f97', 'parent_id': '8837406a-66eb-4b1a-91db-70b2f668850f', 'doc_type': 'child', 'chunk_index': 1, 'batch_index': 1695, 'chunk_size': 29}, page_content='## 必备原料和工具  \n- 西红柿\n- 白砂糖\n- 冰箱'), Document(id='81bca3ee-6d72-4c1d-a6c0-6f93668f9bed', metadata={'主标题': '糖拌西红柿的做法', '二级标题': '操作', 'chunk_id': '78a824b6-6442-4426-8978-20f95b6ad1e6', 'parent_id': '8837406a-66eb-4b1a-91db-70b2f668850f', 'doc_type': 'child', 'chunk_index': 3, 'batch_index': 1697, 'chunk_size': 209}, page_content='## 操作  \n- 用刀将西红柿皮米字型划开\n- 用筷子插入西红柿的菊花，在燃气上转动烤 10 秒（或用开水冲 30 秒），直到西红柿皮卷边\n- 把西红柿的衣服脱光\n- 再西红柿大卸八块（沿纹路切可以更多的留汁水），去掉头部根蒂部，备用\n- 全部切好后，将西红柿在盘子中均匀码一层\n- 撒上白糖，重复上面一步直到全部西红柿放完\n- 放入冰箱冷藏 10 分钟\n- 一盘糖拌西红柿就好了，营养美味，酸甜爽口，夏日解暑又解腻'), Document(id='1e2b056f-95d7-4b0d-8970-ab81da5a3576', metadata={'主标题': '糖拌西红柿的做法', 'chunk_id': '5376899c-40ec-47be-b362-4d63c9b837bb', 'parent_id': '883

检索优化模块详细代码：[retrieval_optimization](rag_modules/retrieval_optimization.py)


### 生成集成模块

整个RAG系统的"大脑"，负责理解用户意图、路由查询类型，并生成高质量的回答。

- 智能查询路由：根据用户查询自动判断是列表查询、详细查询还是一般查询，选择最适合的生成策略。

- 查询重写优化：对模糊不清的查询进行智能重写，提升检索效果。比如将"做菜"重写为"简单易做的家常菜谱"。

多模式生成：

- 列表模式：适用于推荐类查询，返回简洁的菜品列表
- 详细模式：适用于制作类查询，提供分步骤的详细指导
- 基础模式：适用于一般性问题，提供常规回答


#### 初始化LLM

In [ ]:
from langchain_community.chat_models.moonshot import MoonshotChat
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("MOONSHOT_API_KEY")

llm = MoonshotChat(
    model="kimi-k2-0711-preview",
    temperature=0.7,
    max_tokens=2048,
    moonshot_api_key=api_key
)


#### 构建上下文

根据 rag 返回的文档列表，提取食谱内容，构建上下文。

In [ ]:
# 构建上下文

docs = parent_docs

context_parts = []
current_length = 0

for i, doc in enumerate(docs, 1):
    metadata_info = f"【食谱 {i}】"
    if 'dish_name' in doc.metadata:
        metadata_info += f" {doc.metadata['dish_name']}"
    if 'category' in doc.metadata:
        metadata_info += f" | 分类: {doc.metadata['category']}"
    if 'difficulty' in doc.metadata:
        metadata_info += f" | 难度: {doc.metadata['difficulty']}"
            
    # 构建文档文本
    doc_text = f"{metadata_info}\n{doc.page_content}\n"
            
    # 检查长度限制
    if current_length + len(doc_text) > 2000:
        break
            
    context_parts.append(doc_text)
    current_length += len(doc_text)
        
context =  "\n" + "="*50 + "\n".join(context_parts)

# 这里并没有将子文档整合为父文档
print(context)



==================================================【食谱 1】 糖拌西红柿 | 分类: 素菜 | 难度: 简单
# 糖拌西红柿的做法

![示例菜成品](./火山飘雪.jpg)

新鲜可口，制作简便，营养价值高，适合夏季食用，家庭餐桌上的一道美味凉菜。西红柿含有大量的维生素 C, 做法简单 几分钟就可完成。

预估烹饪难度：★★

## 必备原料和工具

- 西红柿
- 白砂糖
- 冰箱

## 计算

一份正好够 2 人吃。

每份：

- 西红柿 2 个（每个西红柿约 100g，共 200g）
- 白砂糖 20g

## 操作

- 用刀将西红柿皮米字型划开
- 用筷子插入西红柿的菊花，在燃气上转动烤 10 秒（或用开水冲 30 秒），直到西红柿皮卷边
- 把西红柿的衣服脱光
- 再西红柿大卸八块（沿纹路切可以更多的留汁水），去掉头部根蒂部，备用
- 全部切好后，将西红柿在盘子中均匀码一层
- 撒上白糖，重复上面一步直到全部西红柿放完
- 放入冰箱冷藏 10 分钟
- 一盘糖拌西红柿就好了，营养美味，酸甜爽口，夏日解暑又解腻

## 附加内容

在制作过程中 请您小心使用刀具。

【食谱 2】 西红柿牛腩 | 分类: 荤菜 | 难度: 非常困难
# 西红柿牛腩的做法

西红柿牛腩汤汁浓厚酸甜可口，牛肉软绵醇香，搭配米饭绝配。一般初学者需要 90 分钟完成。

预估烹饪难度：★★★★★

## 必备原料和工具

* 西红柿
* 牛腩
* 燃气灶（西红柿去皮用）
* 高压锅/砂锅/普通铝锅（铁锅）
* 2cm 两段葱段、两片姜片，葱花、姜各 10g
* 生抽、白胡椒粉，白糖，料/黄酒，八角三小片
* 牛腩（挑选肥瘦相间的口感比较好）

## 计算

每份：

- 西红柿 3-4 个（每个约 200g）
- 牛腩 500g
- 食用油 20-30ml

## 操作

- 牛腩切条、切块成长宽高均 2cm ，冷水下锅，开锅煮制 2 分钟去除血水，捞出冲洗干净
- 另起锅 2L 水烧开，加入 2cm 两段葱段、两片姜片、八角、料/黄酒 5-10ml，放入焯好的牛肉，盖盖炖制（砂锅 1 小时，高压锅炖肉模式 45 分钟），筷子能轻松插透就证明炖好了
- 西红柿去皮：西红柿头部滑十字至腰线，筷子/刀叉从果蒂捅入，煤气灶小火，

#### 生成基础回答

#### 生成分布回答

#### 生成列表回答

#### 查询路由

#### 查询重写优化

等功能，请直接查看 [rag_modules/generation_integration.py](rag_modules/generation_integration.py)
